# EDA — Análise de Renda no Brasil

KPIs e análises que costumam ser cobradas em entrevistas:
- ranking de estados (renda média)
- evolução temporal (nacional e por região)
- desigualdade (dispersão) por UF
- crescimento % ano a ano
- métricas ponderadas por população


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

df = pd.read_csv(Path('../data/renda_brasil_clean.csv'))
df.head()

In [ ]:
# KPIs nacionais (último ano)
latest_year = df['ano'].max()
df_latest = df[df['ano'] == latest_year].copy()

kpi_renda_media_nacional = df_latest['renda_media'].mean()
kpi_estado_top = df_latest.sort_values('renda_media', ascending=False).iloc[0][['uf','renda_media']]
kpi_estado_bottom = df_latest.sort_values('renda_media', ascending=True).iloc[0][['uf','renda_media']]

{
 'ano_ref': int(latest_year),
 'renda_media_nacional': round(float(kpi_renda_media_nacional), 2),
 'top_uf': kpi_estado_top.to_dict(),
 'bottom_uf': kpi_estado_bottom.to_dict(),
}

In [ ]:
# Ranking por UF (último ano)
rank_uf = df_latest.groupby('uf')['renda_media'].mean().sort_values(ascending=False)
rank_uf.head(10)

In [ ]:
# Evolução nacional (média simples)
nat = df.groupby('ano')['renda_media'].mean().sort_index()
plt.figure(figsize=(9,4))
nat.plot(kind='line')
plt.title('Renda Média Nacional (média simples)')
plt.xlabel('Ano')
plt.ylabel('R$')
plt.tight_layout()
plt.show()

In [ ]:
# Evolução por região
reg = df.groupby(['ano','regiao'])['renda_media'].mean().reset_index()
pivot = reg.pivot(index='ano', columns='regiao', values='renda_media').sort_index()
plt.figure(figsize=(10,4))
pivot.plot(kind='line')
plt.title('Renda Média por Região')
plt.xlabel('Ano')
plt.ylabel('R$')
plt.tight_layout()
plt.show()

In [ ]:
# Desigualdade (dispersão) por UF ao longo do tempo
disp = df.groupby('uf')['renda_media'].std().sort_values(ascending=False)
plt.figure(figsize=(10,4))
disp.head(12).plot(kind='bar')
plt.title('Top 12 UFs por variabilidade de renda (desvio padrão)')
plt.xlabel('UF')
plt.ylabel('Desvio padrão (R$)')
plt.tight_layout()
plt.show()
disp.head(12)

In [ ]:
# Crescimento % ano a ano por UF
df_sorted = df.sort_values(['uf','ano']).copy()
df_sorted['crescimento_pct'] = df_sorted.groupby('uf')['renda_media'].pct_change() * 100

growth_latest = df_sorted[df_sorted['ano'] == latest_year][['uf','crescimento_pct']].dropna().sort_values('crescimento_pct', ascending=False)
growth_latest.head(10)

In [ ]:
# Métrica ponderada por população (proxy)
weighted = df.groupby('ano').apply(lambda x: (x['renda_media']*x['populacao']).sum() / x['populacao'].sum()).sort_index()
plt.figure(figsize=(9,4))
weighted.plot(kind='line')
plt.title('Renda Média Nacional (ponderada por população)')
plt.xlabel('Ano')
plt.ylabel('R$')
plt.tight_layout()
plt.show()
weighted